In [1]:
import xml.etree.cElementTree as ET
from pathlib import Path
import os
from tqdm import tqdm
 
def parseXML(xml_file, dst_path):
    tree = ET.ElementTree(file=xml_file)
    root = tree.getroot()

    if not Path(dst_path).exists():
        os.mkdir(dst_path)
    
    for child in root:
        if child.tag == "image":
            image_name = child.attrib['name']
            cls, width, height = 0, int(child.attrib['width']), int(child.attrib['height'])
            pps = []
            for step_child in child:
                if step_child.tag == "points":
                    points = step_child.attrib['points']
                    points = points.replace(';', ',')
                    points = list(map(float, points.split(',')))
                    points = [points[i] / width if i % 2 == 0 else points[i] / height
                               for i in range(len(points))]
                    pps.append([cls, *points])
            with open(dst_path + '/' + image_name.split('.')[0] + '.txt', 'w') as file: 
                for pp in pps:
                    print(*pp, file=file)
if __name__ == "__main__":
    parseXML("/home/anton/PycharmProjects/kaggle_competitions/severstal_aiijc/uglov/annotations.xml",
             'labels_ugols')

In [3]:
from sklearn.model_selection import train_test_split
from glob import glob

images_dir = 'uglov/ugols'
labels_dir = 'labels_ugols'

dst_dir = 'ugols_seg'

if not Path(dst_dir).exists():
    os.mkdir(dst_dir)

images_path_ = list(sorted(glob(images_dir + '/*')))
labels_path = list(sorted(glob(labels_dir + '/*')))

images_path = [image_path for image_path in images_path_ if labels_dir + '/' + image_path.split('/')[-1].split('.')[0] + '.txt' in labels_path]
    
images_path_train, images_path_valid, labels_path_train, labels_path_valid = train_test_split(
    images_path, labels_path, test_size=0.2, random_state=69
)
if not Path(dst_dir + '/train').exists():
    os.mkdir(dst_dir + '/train')
    
if not Path(dst_dir + '/valid').exists():
    os.mkdir(dst_dir + '/valid')
        
if not Path(dst_dir + '/train/images').exists():
    os.mkdir(dst_dir + '/train/images')
    
if not Path(dst_dir + '/valid/images').exists():
    os.mkdir(dst_dir + '/valid/images')
    
if not Path(dst_dir + '/train/labels').exists():
    os.mkdir(dst_dir + '/train/labels')

if not Path(dst_dir + '/valid/labels').exists():
    os.mkdir(dst_dir + '/valid/labels')
        
for image_path, label_path in zip(images_path_train, labels_path_train):
    os.system(f'cp {image_path} {dst_dir}/train/images/{image_path.split("/")[-1]}')
    os.system(f'cp {label_path} {dst_dir}/train/labels/{label_path.split("/")[-1]}')
        
for image_path, label_path in zip(images_path_valid, labels_path_valid):
    os.system(f'cp {image_path} {dst_dir}/valid/images/{image_path.split("/")[-1]}')
    os.system(f'cp {label_path} {dst_dir}/valid/labels/{label_path.split("/")[-1]}')
